# ID2214/FID3214 Assignment 3 Group no. [enter]
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
import math
from sklearn import tree

## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def accuracy(dataframe, correctlabels):
    
    df = dataframe.copy()
    labels = df.idxmax(axis=1)
    truelabels = (labels == correctlabels).sum(axis=0)
    accuracy = truelabels/len(df)
    return accuracy


def brier_score(df, correctlabels):
    score = 0  
    observed_probs = pd.get_dummies(correctlabels)
    score = (df - observed_probs) ** 2
    score = score.sum(axis=1)
    score = score.mean()
        
    return score

def calc_auc(predictions,correctlabels,col):
    pos, neg =get_pos_and_neg(predictions,correctlabels,col)
    pos=pos.tolist()
    neg=neg.tolist()
    tot_tp=0
    tot_fp=0
    for i in range(len(pos)):
        tot_tp=pos[i]+tot_tp
        tot_fp=tot_fp+neg[i]
        
    a=0
    cov_tp=0
    for i in range(len(pos)):
        if neg[i]==0 :
            cov_tp=cov_tp+pos[i]
        elif pos[i]==0 :
            a=a+(cov_tp/tot_tp)*(neg[i]/tot_fp)
        else:
            a=a+(cov_tp/tot_tp)*(neg[i]/tot_fp)+(pos[i]/tot_tp)*(neg[i]/tot_fp)/2
            cov_tp=cov_tp+pos[i]
    return a

def auc(predictions,correctlabels):
    class_frequency = dict(pd.Series(correctlabels).value_counts(normalize = True))
    x=0
    
    for col in predictions.columns:
        x=x+calc_auc(predictions,correctlabels,col)*class_frequency[col]
    return x 

def get_pos_and_neg(predictions,correctlabels,pos_label):
    scores={}
    
    for i in range(len(predictions)):
        score = predictions.loc[i,pos_label]
        (p,n) = scores.get(score,(0,0))
        if correctlabels[i] == pos_label:
            scores[score] = (p+1,n)
        else:
            scores[score] = (p,n+1)
    scores = np.array([[score,scores[score][0],scores[score][1]] for score in scores.keys()])
    #display(scores)
    scores = scores[np.argsort(scores[:,0])[::-1]]
    pos = scores[:,1]
    neg = scores[:,2]
    #display(pos,neg)
    return pos, neg

def create_bins(glass_train_df,nobins,bintype):
    df2=glass_train_df.copy()
    binning=dict()
    for col in df2.columns:
        if(col not in['CLASS','ID'] and df2.dtypes[col] in ['float64','int64','float32','int32']):
            if(bintype=='equal-size'):
                df2[col],bins=pd.qcut(df2[col],q=10,retbins=True,labels=False,duplicates='drop')
            elif(bintype=='equal-width'):
                df2[col],bins=pd.cut(df2[col],10,retbins=True,labels=False,duplicates='drop')
            df2[col]=df2[col].astype('category')
            df2[col]=df2[col].cat.set_categories([str(i) for i in df2[col].cat.categories],rename=True)
            bins[0]=-np.inf
            bins[-1]=np.inf
            binning[col]=bins
    return df2,binning

def apply_bins(dataframe,binning):
    
    df = dataframe.copy()
    bin_labels = {}
    for col in binning:  
        bins = binning[col]
        df[col] = pd.cut(df[col],bins,labels=False)
        df[col] = df[col].astype("category")
        df[col] = df[col].cat.set_categories([str(i) for i in df[col].cat.categories], rename = True)        
    df = df.astype("category")
    return df

def create_column_filter(df):
    df1=df.copy()
    #display(df1)
    rem=list()
    for col in df1.columns:
        if col not in ['CLASS','ID'] and df1[col].nunique()<=1:
        
            df1=df1.drop(columns=col)
        else:
            rem.append(col)
    return df1,rem 

def create_imputation(dataframe):
    df = dataframe.copy()
    imputation = {}
    
    #select numerical and categorical columns
    numeric_column = df.select_dtypes(include=["int64", "float64"]).columns
    categorical_column = df.select_dtypes(exclude=["int64", "float64"]).columns
    #exclude class and id column 
    numeric_column = [column for column in numeric_column if column != "ID" if column != "CLASS"]
    categorical_column = [column for column in categorical_column if column != "ID" if column != "CLASS"]
    
    for column in df.columns:
        if (column not in ['CLASS','ID'] and df.dtypes[column] in ['float64','int64','float32','int32']):
            if df[column].any():
                imputation[column] = df[column].mean()
                df[column]=df[column].fillna(df[column].mean())
                
            #Replace whole column with Nan to 0
            else:
                df[column]=df[column].fillna(0)
                imputation[column] = 0
                
        elif (column not in ['CLASS','ID'] and df.dtypes[column] in ['object','category']):
            if df[column].any():
                imputation[column] = df[column].mode()[0]
                df[column]=df[column].fillna(df[column].mode()[0])
                
            #Replace whole column with Nan to ""
            else:
                imputation[column] = ""
                df[column]=df[column].fillna("")
                
    
    return df, imputation

def create_one_hot(df):
    dfCopy = df.copy()
    one_hot = {}
    
    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and not pd.api.types.is_numeric_dtype(dfCopy[col]):

            dfCopy[col] = dfCopy[col].astype("category")
            one_hot[col]= list(dfCopy[col].cat.categories)

            y = pd.get_dummies(dfCopy[col], prefix = col, dtype=float)
            dfCopy = pd.concat([dfCopy, y], axis=1)
            
            dfCopy.drop(col, inplace = True, axis = 1)
            
    return dfCopy, one_hot


def apply_one_hot(df, one_hot):
    dfCopy = df.copy()
    
    for col in one_hot:
        y = pd.get_dummies(dfCopy[col], dtype=float)
        for cat in one_hot[col]:
            if cat in y:
                dfCopy[col+'-'+str(cat)] = y[cat]
            else:
                dfCopy[col+'-'+str(cat)] = 0
    
        dfCopy.drop(col, inplace = True, axis = 1)

    return dfCopy

def create_normalization(df,normalizationtype='minmax'):
    new_df=df.copy()
    
    colss = new_df.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    colss = colss.select_dtypes(include=[np.int_, np.float_]) 
    
    cols=colss.columns.tolist()

    if normalizationtype=='minmax':
        mins=new_df[cols].min()
        maxs=new_df[cols].max()
        types=[normalizationtype]*len(cols)
        minmax=zip(types,mins,maxs)
        normalization=dict(zip(cols,minmax))
        new_df[cols]=(new_df[cols]-mins)/(maxs-mins)
    
    if normalizationtype=='zscore':
        means=new_df[cols].mean()
        stds=new_df[cols].std()
        types=[normalizationtype]*len(cols)
        zscore=zip(types,means,stds)
        normalization=dict(zip(cols,zscore))
        new_df[cols]=(new_df[cols]-means)/(stds)   
    
    return new_df,normalization

def apply_imputation(dataframe,imputation):
    df = dataframe.copy()
    
    for column in imputation.keys():
        if column not in ['CLASS','ID'] :
            df[column]=df[column].fillna(imputation[column])
    
    return df

def apply_normalization(df, normalization):
    new_df = df.copy()
    cols = new_df.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    cols = cols.select_dtypes(include=[np.int_, np.float_])
    
    colss = cols.columns.tolist()
    
    for col in colss:
        if normalization[col][0] == "minmax":
            new_df[col] = np.clip([(x - normalization[col][1])/(normalization[col][2]-normalization[col][1]) for x in df[col]], 0, 1) # limit the output range to 0,1
            
        elif normalization[col][0] == "zscore":
            new_df[col] = [(x - normalization[col][1])/normalization[col][2] for x in df[col]]
    
    return new_df

def apply_column_filter(df,column_filter):
    df1=df.copy()
    df1=df1[df1.columns.intersection(column_filter)]
    return df1

## 1. Define the class RandomForest

In [3]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>

class RandomForest:
    def __init__(self):
        self.column_filter=None
        self.imputation=None
        self.one_hot=None
        self.labels=None
        self.model=None


    def fit(self,df,no_trees=100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        #display(df)
        df["CLASS"] = df["CLASS"].astype("category")
        y=(df['CLASS'].to_numpy())
        #display(y)
       # self.labels = list(df["CLASS"].cat.categories)
        ##self.labels=np.sort(self.labels)
        self.labels=np.sort(df.CLASS).tolist()
        df=df.drop(['CLASS'],axis='columns')
        X=(df.to_numpy())
        num_rows, num_cols = X.shape
        lg=np.log2(num_cols)
        lg=(math.floor(lg))
        self.model=list()
        
        for i in range(no_trees):
            #sample=df.sample(n=len(df),replace=True)
            #Ay=sample['CLASS']
            #Ax=sample.drop(columns=['CLASS'])
            dt=tree.DecisionTreeClassifier(max_features=lg)
            index = np.random.choice(X.shape[0], num_rows, replace=True)
            x_random = X[index]
            y_random = y[index]
            dt.fit(x_random,y_random)
            #display(dt)
            self.model.append(dt)
            
    
    
    
    
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#

    def predict(self,df):
        df=df.drop(['CLASS'],axis='columns')
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        
        num_rows, num_cols = df.shape
        z=np.zeros((num_rows,len(np.unique(self.labels))))
        mp=dict()
        
        for t in range(len(self.model)):
            pr=(self.model[t].predict_proba(df))
            #display(pr)
            z=z+pr
            #display(z)
        #display(z)
        z=(z/len(self.model))
        df2 = pd.DataFrame(z, columns = np.unique(self.labels))
        #display(df2)
        return df2

# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 



In [5]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.14 s.
Testing time: 0.20 s.
Accuracy: 0.9123
AUC: 0.9923
Brier score: 0.1753


In [6]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0199


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [63]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
#

class RandomForest:
    def __init__(self):
        self.column_filter=None
        self.imputation=None
        self.one_hot=None
        self.labels=None
        self.model=None


        
        
        
    def fit(self,df,no_trees=100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        #display(df)
        df["CLASS"] = df["CLASS"].astype("category")
        y=(df['CLASS'].to_numpy())
        #display(y)
        self.labels = list(df["CLASS"].cat.categories)
        ##self.labels=np.sort(self.labels)
        #self.labels=np.sort(df.CLASS).tolist()
        df=df.drop(['CLASS'],axis='columns')
        X=(df.to_numpy())
        num_rows, num_cols = X.shape
        lg=np.log2(num_cols)
        lg=(math.floor(lg))
        self.model=list()
        
        for i in range(no_trees):
            #sample=df.sample(n=len(df),replace=True)
            #Ay=sample['CLASS']
            #Ax=sample.drop(columns=['CLASS'])
            dt=tree.DecisionTreeClassifier(max_features=lg)
            index = np.random.choice(X.shape[0], num_rows, replace=True)
            x_random = X[index]
            y_random = y[index]
            dt.fit(x_random,y_random)
            #display(dt)
            self.model.append(dt)
            
    
    



    def predict(self,df):
        df=df.drop(['CLASS'],axis='columns')
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        mp=dict()
        display(self.labels)
        for i in range(len(np.unique(self.labels))):
            mp[self.labels[i]]=i
        num_rows, num_cols = df.shape
        z=np.zeros((num_rows,len(np.unique(self.labels))))
        display(mp)
        for t in range(len(self.model)):
            
            pr=(self.model[t].predict_proba(df))
            #display(self.model[t].)
            h=np.zeros((num_rows,len(np.unique(self.labels))))
            for j in range(len(pr[0])):
                #h[:,1]=h[:,1]+pr[:,j]
                #display(j)
                #display(mp[self.model[t].classes_[j]])
                
                #h[:,mp[self.model[t].classes_[j]]]=h[:,mp[self.model[t].classes_[j]]]+pr
                h[:,mp[self.model[t].classes_[j]]]=h[:,mp[self.model[t].classes_[j]]]+pr[:,j]
                #display(h)
            #display(self.model[t].classes_[0])
            #display(pr)
            z=z+h
            #display(z)
        #display(z)
        z=(z/len(self.model))
        df2 = pd.DataFrame(z, columns = np.unique(self.labels))
        #display(df2)
        return df2






# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



In [64]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.17 s.


['1', '2', '3', '5', 'U']

{'1': 0, '2': 1, '3': 2, '5': 3, 'U': 4}

Testing time: 0.21 s.
Accuracy: 0.9443
AUC: 0.9722
Brier score: 0.1006


## 2b. Estimate predictive performance using out-of-bag predictions

In [74]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:

class RandomForest:
    def __init__(self):
        self.column_filter=None
        self.imputation=None
        self.one_hot=None
        self.labels=None
        self.model=None
        self.oob_acc=None

    
            
    def fit(self,df,no_trees=100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        #display(df)
        df["CLASS"] = df["CLASS"].astype("category")
        y=(df['CLASS'].to_numpy())
        ydf= df["CLASS"]
        #display(y)
        self.labels = list(df["CLASS"].cat.categories)
        ##self.labels=np.sort(self.labels)
        #self.labels=np.sort(df.CLASS).tolist()
        df=df.drop(['CLASS'],axis='columns')
        X=(df.to_numpy())
        num_rows, num_cols = X.shape
        lg=np.log2(num_cols)
        lg=(math.floor(lg))
        self.model=list()
        count=np.zeros(len(df))
        z=np.zeros((len(df),len(self.labels)))
        for i in range(no_trees):
            #sample=df.sample(n=len(df),replace=True)
            #Ay=sample['CLASS']
            #Ax=sample.drop(columns=['CLASS'])
            dt=tree.DecisionTreeClassifier(max_features=lg)
            index = np.random.choice(X.shape[0], num_rows, replace=True)
            x_random = X[index]
            y_random = y[index]
            
            
            dt.fit(x_random,y_random)
            notinindex=list()
            for j in range(num_rows):
                if(j not in index):
                    notinindex.append(j)
                    #display(df.iloc[j,:])
            
            dfz=df.loc[df.index.isin(notinindex)]
            pr=dt.predict_proba(dfz)
            j=list()
            for i,row in dfz.iterrows():
                j.append(i)
                
            for x in range(len(dfz)):
                val=list();
                for i in range(len(pr[0])):
                    val.append(self.labels.index(dt.classes_[i]))
                z[j[x],val]+=pr[x]
            count[dfz.index]+=1
        
        z=z / count[:, None]
        df2=pd.DataFrame(z,columns=np.unique(self.labels))
            
        indexes=np.unique(df2.index)
            #display(df2)
        observed=ydf.loc[ydf.index.isin(indexes)]
        self.oob_acc=accuracy(df2,observed)
        self.model.append(dt)
        #display(self.oob_acc)
                
            
            #display(y_random)
            #display(z_random)
            
            
            #dfz=z_random[None,:]
            #display(dfz)
            #display(dt)
            #display(df)
            #display(dfz)
            
            #dfz=df.drop(['CLASS'],axis='columns')
            #dfz = apply_column_filter(df, self.column_filter)
            #dfz = apply_imputation(df, self.imputation)
            #dfz = apply_one_hot(df,self.one_hot)
            #display(pr)
            #pr=dt.predict_proba(dfz)
            #self.model.append(dt)
    
    
        
        



# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#

# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector


        
    def predict(self,df):
        df=df.drop(['CLASS'],axis='columns')
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        mp=dict()
        display(self.labels)
        for i in range(len(np.unique(self.labels))):
            mp[self.labels[i]]=i
        num_rows, num_cols = df.shape
        f=np.zeros((num_rows,len(np.unique(self.labels))))
        display(mp)
        for t in range(len(self.model)):
            
            pr=(self.model[t].predict_proba(df))
            #display(self.model[t].)
            h=np.zeros((num_rows,len(np.unique(self.labels))))
            for j in range(len(pr[0])):
                #h[:,1]=h[:,1]+pr[:,j]
                #display(j)
                #display(mp[self.model[t].classes_[j]])
                
                #h[:,mp[self.model[t].classes_[j]]]=h[:,mp[self.model[t].classes_[j]]]+pr
                h[:,mp[self.model[t].classes_[j]]]=h[:,mp[self.model[t].classes_[j]]]+pr[:,j]
                #display(h)
            #display(self.model[t].classes_[0])
            #display(pr)
            f=f+h
            #display(z)
        #display(f)
        #display(len(self.model))
        f=(f/len(self.model))
        df2 = pd.DataFrame(f, columns = np.unique(self.labels))
        #display(df2)
        return df2


In [75]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()
rf.fit(train_df)
#predictions = rf.predict(test_df)

In [76]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.41 s.
OOB accuracy: 0.9510


['1', '2', '3', '5', 'U']

{'1': 0, '2': 1, '3': 2, '5': 3, 'U': 4}

Testing time: 0.04 s.
Accuracy: 0.8664
AUC: 0.8337
Brier score: 0.2673


In [77]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

['1', '2', '3', '5', 'U']

{'1': 0, '2': 1, '3': 2, '5': 3, 'U': 4}

Accuracy on training set: 0.96
AUC on training set: 0.95
Brier score on training set: 0.08


### Comment on assumptions, things that do not work properly, etc.